In [44]:
import pandas as pd
import numpy as np
import re
import random

In [2]:
from linkedlist import LinkedList
from collections import OrderedDict
import math

In [3]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [4]:
from tqdm import tqdm

In [5]:
filename = './data/input_corpus.txt'
df = pd.read_csv(filename, sep='\t',names=['DocID','content'])
df.shape

(5000, 2)

In [ ]:
def preprocess(text):
    
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()

    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.strip()
    text = ' '.join(text.split())
    
    tokens = text.split(' ')
    filtered_tokens = [token for token in tokens if token not in stop_words]
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    
    stemmed_text = ' '.join(stemmed_tokens)
    return stemmed_tokens

In [6]:
class Preprocessor:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.ps = PorterStemmer()

    def get_doc_id(self, doc):
        """ Splits each line of the document, into doc_id & text.
            Already implemented"""
        arr = doc.split("\t")
        return int(arr[0]), arr[1]

    def tokenizer(self, text):
        """ Implement logic to pre-process & tokenize document text.
            Write the code in such a way that it can be re-used for processing the user's query.
            To be implemented."""
        text = text.lower()
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        text = text.strip()
        text = ' '.join(text.split())

        tokens = text.split(' ')
        filtered_tokens = [token for token in tokens if token not in self.stop_words]
        stemmed_tokens = [self.ps.stem(token) for token in filtered_tokens]

        stemmed_text = ' '.join(stemmed_tokens)
        return stemmed_tokens

In [7]:
p = Preprocessor()
p

In [8]:
text = 'Epidemiological and clinical characteristics of 136 cases of COVID-19 in main district of Chongqing'
p.tokenizer(text)

['epidemiolog',
 'clinic',
 'characterist',
 '136',
 'case',
 'covid19',
 'main',
 'district',
 'chongq']

In [9]:
class Node:
    def __init__(self, value=None, next=None, skip_next=None, tf=0):
        """ Class to define the structure of each node in a linked list (postings list).
            Value: document id, Next: Pointer to the next node
            Add more parameters if needed.
            Hint: You may want to define skip pointers & appropriate score calculation here"""
        self.value = value #document-ID
        self.next = next #next pointer
        self.skip_next = skip_next #skip pointer
        self.tf = tf #tf score for each term,doc pair
        
    def increase_tf_count(self):
        self.tf = self.tf+1

In [10]:
n = Node(5)
n.value

5

In [11]:
n.next

In [12]:
m = Node(6,n)
m.value

6

In [13]:
m

In [14]:
class Node:

    def __init__(self, value=None, next=None, skip_next=None, tf=0):
        """ Class to define the structure of each node in a linked list (postings list).
            Value: document id, Next: Pointer to the next node
            Add more parameters if needed.
            Hint: You may want to define skip pointers & appropriate score calculation here"""
        self.value = value #document-ID
        self.next = next #next pointer
        self.skip_next = skip_next #skip pointer
        self.tf = tf #tf score for each term,doc pair
        
    def increase_tf_count(self):
        self.tf = self.tf+1


class LinkedList:
    """ Class to define a linked list (postings list). Each element in the linked list is of the type 'Node'
        Each term in the inverted index has an associated linked list object.
        Feel free to add additional functions to this class."""
    def __init__(self):
        self.start_node = None #start of PL
        self.end_node = None #end of PL
        self.length, self.n_skips = 0, 0 #length of PL, no of skips, term freq
        self.skip_length = None #skip length
        
    def traverse_list(self):
        
        if self.start_node is None:
            return ["List is empty"]
        
            """ Write logic to traverse the linked list.
                To be implemented."""
        traversal = []
        tf_scores = []
        curr = self.start_node
        traversal.append(curr.value)
        tf_scores.append(curr.tf)
        while(curr.next):
            traversal.append(curr.next.value)
            tf_scores.append(curr.next.tf)
            curr = curr.next
        return traversal,tf_scores

    def traverse_skips(self):
        traversal = []
        if self.start_node is None:
            return
        else:
            """ Write logic to traverse the linked list using skip pointers.
                To be implemented."""
            curr = self.start_node
            if(curr.skip_next):
                traversal.append(curr.value)
            while(curr.skip_next):
                traversal.append(curr.skip_next.value)
                curr = curr.skip_next
            return traversal

    def add_skip_connections(self):
        if(self.length < 3):
            return
        
        """ Write logic to add skip pointers to the linked list. 
            This function does not return anything.
            To be implemented."""
        
        n_skips = math.floor(math.sqrt(self.length))
        if n_skips * n_skips == self.length:
            n_skips = n_skips - 1
        
        self.n_skips = n_skips
        self.skip_length = int(np.round(math.sqrt(self.length),0))
        
        prev = None
        curr = self.start_node
        while(curr):
            count = 0
            prev = curr
            while((curr.next is not None) & (count < self.skip_length)):
                curr = curr.next
                count += 1
            if(count == self.skip_length):
                prev.skip_next = curr 
            if(curr.next is None):
                break

    def insert(self, value):
        """ Write logic to add new elements to the linked list.
            Insert the element at an appropriate position, such that elements to the left are lower than the inserted
            element, and elements to the right are greater than the inserted element.
            To be implemented. """
        new_node = Node(value,None,None,1)
        prev_node = None
        curr_node = self.start_node
        
        if(self.length == 0 or curr_node is None):
            self.start_node = self.end_node = new_node
        else:
            while((curr_node.next is not None) & (curr_node.value < value)):
                prev_node = curr_node
                curr_node = curr_node.next
            if(curr_node.value > value):
                if(prev_node is None):
                    self.start_node = new_node
                    self.start_node.next = curr_node
                else:
                    prev_node.next = new_node
                    new_node.next = curr_node
            elif(curr_node.value == value):
                curr_node.increase_tf_count()
                self.length -= 1
            else :
                new_node.next = curr_node.next
                curr_node.next = new_node
                if(new_node.next is None):
                    self.end_node = new_node
        self.length += 1  



In [15]:
ll = LinkedList()
ll

In [17]:
ll = LinkedList()
elms_to_insert = [7,8,9]
for el in elms_to_insert:
    ll.insert(el)
ll.add_skip_connections()
ll.traverse_skips()

[7, 9]

In [19]:
elms_to_insert = [10,15,5,1,25,12,12,14,15,35,35,30,50]
for el in elms_to_insert:
    ll.insert(el)
    print(el,ll.traverse_list(), ll.start_node.value,ll.end_node.value, ll.length)

10 ([7, 8, 9, 10], [1, 1, 1, 1]) 7 10 4
15 ([7, 8, 9, 10, 15], [1, 1, 1, 1, 1]) 7 15 5
5 ([5, 7, 8, 9, 10, 15], [1, 1, 1, 1, 1, 1]) 5 15 6
1 ([1, 5, 7, 8, 9, 10, 15], [1, 1, 1, 1, 1, 1, 1]) 1 15 7
25 ([1, 5, 7, 8, 9, 10, 15, 25], [1, 1, 1, 1, 1, 1, 1, 1]) 1 25 8
12 ([1, 5, 7, 8, 9, 10, 12, 15, 25], [1, 1, 1, 1, 1, 1, 1, 1, 1]) 1 25 9
12 ([1, 5, 7, 8, 9, 10, 12, 15, 25], [1, 1, 1, 1, 1, 1, 2, 1, 1]) 1 25 9
14 ([1, 5, 7, 8, 9, 10, 12, 14, 15, 25], [1, 1, 1, 1, 1, 1, 2, 1, 1, 1]) 1 25 10
15 ([1, 5, 7, 8, 9, 10, 12, 14, 15, 25], [1, 1, 1, 1, 1, 1, 2, 1, 2, 1]) 1 25 10
35 ([1, 5, 7, 8, 9, 10, 12, 14, 15, 25, 35], [1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1]) 1 35 11
35 ([1, 5, 7, 8, 9, 10, 12, 14, 15, 25, 35], [1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2]) 1 35 11
30 ([1, 5, 7, 8, 9, 10, 12, 14, 15, 25, 30, 35], [1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2]) 1 35 12
50 ([1, 5, 7, 8, 9, 10, 12, 14, 15, 25, 30, 35, 50], [1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1]) 1 50 13


In [20]:
p = ll.start_node
while(p):
    print(p.value, p.tf)
    p = p.next

1 1
5 1
7 1
8 1
9 1
10 1
12 2
14 1
15 2
25 1
30 1
35 2
50 1


In [21]:
ll.length

13

In [22]:
len(elms_to_insert)

13

In [23]:
ll.traverse_list()

([1, 5, 7, 8, 9, 10, 12, 14, 15, 25, 30, 35, 50],
 [1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1])

In [24]:
ll.add_skip_connections()
ll.traverse_skips()

[1, 9, 15, 50]

In [25]:
ll.length

13

In [26]:
class Indexer:
    def __init__(self):
        """ Add more attributes if needed"""
        self.inverted_index = OrderedDict({}) #contains key:term, value:(length of PL, PL)
        self.doc_token_counts = OrderedDict({}) #no of tokens in a document 
        self.tf_idf_scores = OrderedDict({}) #tf-idf scores for each (term,doc) pair
        
    def get_index(self):
        """ Function to get the index.
            Already implemented."""
        return self.inverted_index
    
    def get_doc_token_counts(self):
        """ Function to get the index.
            Already implemented."""
        return self.doc_token_counts

    def get_tf_idf_scores(self):
        """ Function to get the index.
            Already implemented."""
        return self.tf_idf_scores
    
    def generate_inverted_index(self, doc_id, tokenized_document):
        """ This function adds each tokenized document to the index. This in turn uses the function add_to_index
            Already implemented."""
        for term_ in tokenized_document:
            self.add_to_index(term_, doc_id)
        self.doc_token_counts[doc_id] = len(tokenized_document)

    def add_to_index(self, term_, doc_id_):
        
        """ This function adds each term & document id to the index.
            If a term is not present in the index, then add the term to the index & initialize a new postings list (linked list).
            If a term is present, then add the document to the appropriate position in the posstings list of the term.
            To be implemented."""
        if term_ not in self.inverted_index.keys():
            self.inverted_index[term_] = [0,LinkedList()]
            
        lpl = self.inverted_index[term_][0]
        self.inverted_index[term_][0] = lpl + 1
        self.inverted_index[term_][1].insert(doc_id_)

    def sort_terms(self):
        """ Sorting the index by terms.
            Already implemented."""
        sorted_index = OrderedDict({})
        for k in sorted(self.inverted_index.keys()):
            sorted_index[k] = self.inverted_index[k]
        self.inverted_index = sorted_index

    def add_skip_connections(self):
        """ For each postings list in the index, add skip pointers.
            To be implemented."""
        for term_ in self.inverted_index.keys():
            self.inverted_index[term_][1].add_skip_connections()

    def calculate_tf(self,n_term,n_doc):
        return n_term/n_doc
            
    def calculate_idf(self, D, N):
        if D == 0:
            return 0 
        return math.log(N / D)
            
    def calculate_tf_idf(self):
        """ Calculate tf-idf score for each document in the postings lists of the index.
            To be implemented."""
        tf_idf_scores = {}
        N = len(self.doc_token_counts)
        for term_ in self.inverted_index.keys():
            D = self.inverted_index[term_][0]
            idf_score = self.calculate_idf(D,N)
    
            PL,tf_counts = self.inverted_index[term_][1].traverse_list() 
            for i in range(len(PL)):
                n_term =  tf_counts[i]
                n_doc = self.doc_token_counts[PL[i]]
                tf_score = self.calculate_tf(n_term,n_doc)
                tf_idf_scores[(term_,PL[i])] = np.round(tf_score * idf_score, 2)
        self.tf_idf_scores = tf_idf_scores


In [27]:
indexer = Indexer()
indexer

In [28]:
indexer.get_index()

OrderedDict()

In [29]:
indexer.generate_inverted_index(1, ['t1','t2','t3'])
indexer.generate_inverted_index(2, ['t1','t1','t2'])

In [30]:
indexer.inverted_index

OrderedDict([('t1', [3, <__main__.LinkedList at 0x7f7d80285b80>]),
             ('t2', [2, <__main__.LinkedList at 0x7f7d80285640>]),
             ('t3', [1, <__main__.LinkedList at 0x7f7d80285dc0>])])

In [31]:
indexer.inverted_index['t1'][1].traverse_list()

([1, 2], [1, 2])

In [32]:
indexer.doc_token_counts

OrderedDict([(1, 3), (2, 3)])

In [33]:
indexer.calculate_tf_idf()

In [34]:
def merge(pl_1, pl_2):
    i,j,m,n = 0,0,len(pl_1),len(pl_2)
    pl_merged = []
    while((i < m) & (j < n)):
        if(pl_1[i] == pl_2[j]):
            pl_merged.append(pl_1[i])
            i += 1
            j += 1
        elif(pl_1[i] < pl_2[j]):
            i += 1
        else:
            j += 1
    return pl_merged

In [35]:
a = [1,2]
b = [2,3]
merge(a,b)

[2]

In [36]:
def daat_and(terms):
    '''
    terms = list(set(terms))
    PL_values,PL_counts = [],[]
    for term in terms:
        index = indexer.get_index()
        PL = [i for i in range(10)]#index[term].traverse_list()
        PL_values.append(PL)
        PL_counts.append(len(PL))
    '''
    PL_counts = [5,4,6,2,3]
    PL_values = [[1,2,3,4,5],
                 [4,5,6,10],
                 [1,3,4,7,10,16],
                 [1,4],
                 []
                ]
    
    
    all_PL = list(zip(PL_counts, PL_values))
    all_PL_sorted = sorted(all_PL, key=lambda x: x[0])
    PL_counts, PL_values  = zip(*all_PL_sorted)
    
    min_PL = PL_values[0]
    for i in range(1,len(PL_values)):
        min_PL = merge(min_PL, PL_values[i])
        
    return min_PL

In [37]:
terms = ['t1','t2','t3']
daat_and(terms)

[]

In [53]:
class ProjectRunner:
    def __init__(self):
        self.preprocessor = Preprocessor()
        self.indexer = Indexer()

    def _merge(self, pl_1, pl_2):
        """ Implement the merge algorithm to merge 2 postings list at a time.
            Use appropriate parameters & return types.
            While merging 2 postings list, preserve the maximum tf-idf value of a document.
            To be implemented."""
        i,j,m,n = 0,0,len(pl_1),len(pl_2)
        pl_merged = []
        num_comparisions = 0
        while((i < m) & (j < n)):
            num_comparisions += 1
            if(pl_1[i] == pl_2[j]):
                pl_merged.append(pl_1[i])
                i += 1
                j += 1
            elif(pl_1[i] < pl_2[j]):
                i += 1
            else:
                j += 1
        return pl_merged,num_comparisions

    def _daat_and(self, terms, score=0):
        """ Implement the DAAT AND algorithm, which merges the postings list of N query terms.
            Use appropriate parameters & return types.
            To be implemented."""
        terms = list(set(terms))
        PL_values,PL_counts = [],[]
        for term in terms:
            PL,_ = self._get_postings(term)
            PL_values.append(PL)
            PL_counts.append(len(PL))
            
        all_PL = list(zip(PL_counts, PL_values))
        all_PL_sorted = sorted(all_PL, key=lambda x: x[0])
        PL_counts, PL_values  = zip(*all_PL_sorted)

        min_PL = PL_values[0]
        total_comparisions = 0
        for i in range(1,len(PL_values)):
            min_PL,num_comparisions = self._merge(min_PL, PL_values[i])
            total_comparisions += num_comparisions

        if((score) & (len(min_PL) > 1)):
            doc_scores = []
            for doc in min_PL:
                total_score = 0
                for term in terms:
                    s = self.indexer.get_tf_idf_scores()[(term,doc)]
                    total_score += s 
                doc_scores.append(total_score)
            answer_score = list(zip(min_PL, doc_scores))
            answer_score_sorted = sorted(answer_score, key=lambda x: x[1],reverse=True)
            answer_score, doc_scores  = zip(*answer_score_sorted)  
            return answer_score,total_comparisions
    
        return min_PL,total_comparisions


    def _get_postings(self,term):
        """ Function to get the postings list of a term from the index.
            Use appropriate parameters & return types.
            To be implemented."""
        return self.indexer.get_index()[term][1].traverse_list()

    def _merge_skip(self, pl_1, pl_2):
        """ Implement the merge algorithm to merge 2 postings list at a time.
            Use appropriate parameters & return types.
            While merging 2 postings list, preserve the maximum tf-idf value of a document.
            To be implemented."""
        pl_merged = []
        num_comparisions = 0
        while((pl_1 is not None) & (pl_2 is not None)):
            num_comparisions += 1
            if(pl_1.value == pl_2.value):
                pl_merged.append(pl_1.value)
                pl_1 = pl_1.next
                pl_2 = pl_2.next
                
            elif(pl_1.value < pl_2.value):
                if(pl_1.skip_next is not None) and (pl_1.skip_next.value <= pl_2.value):
                    while(pl_1.skip_next is not None) and (pl_1.skip_next.value <= pl_2.value):
                        pl_1 = pl_1.skip_next
                else:
                    pl_1 = pl_1.next
                  
            else:
                if(pl_2.skip_next is not None) and (pl_2.skip_next.value <= pl_1.value):  
                    while(pl_2.skip_next is not None) and (pl_2.skip_next.value <= pl_1.value):
                        pl_2 = pl_2.skip_next
                else:
                    pl_2 = pl_2.next
                    
        return pl_merged,num_comparisions
    
    def _daat_and_skip(self, terms, score=0):
        """ Implement the DAAT AND algorithm, which merges the postings list of N query terms.
            Use appropriate parameters & return types.
            To be implemented."""
        terms = list(set(terms))
        start_ptrs,PL_counts = [],[]
        for term in terms:
            p = self.indexer.get_index()[term][1].start_node
            l = self.indexer.get_index()[term][1].length
            start_ptrs.append(p)
            PL_counts.append(l)
            
        all_PL = list(zip(PL_counts, start_ptrs))
        all_PL_sorted = sorted(all_PL, key=lambda x: x[0])
        PL_counts, start_ptrs  = zip(*all_PL_sorted)

        min_start_ptr = start_ptrs[0]
        total_comparisions = 0
        for i in range(1,len(start_ptrs)):
            answer,num_comparisions = self._merge_skip(min_start_ptr, start_ptrs[i])
            total_comparisions += num_comparisions
            
        if((score) & (len(answer) > 1)):
            doc_scores = []
            for doc in answer:
                total_score = 0
                for term in terms:
                    total_score += self.indexer.get_tf_idf_scores()[(term,doc)]
                doc_scores.append(total_score)
            answer_score = list(zip(answer, doc_scores))
            answer_score_sorted = sorted(answer_score, key=lambda x: x[1],reverse=True)
            answer_score, doc_scores  = zip(*answer_score_sorted) 
            return answer_score,total_comparisions

        return answer,total_comparisions
    

    def _output_formatter(self, op):
        """ This formats the result in the required format.
            Do NOT change."""
        if op is None or len(op) == 0:
            return [], 0
        op_no_score = [int(i) for i in op]
        results_cnt = len(op_no_score)
        return op_no_score, results_cnt

    def run_indexer(self, corpus):
        """ This function reads & indexes the corpus. After creating the inverted index,
            it sorts the index by the terms, add skip pointers, and calculates the tf-idf scores.
            Already implemented, but you can modify the orchestration, as you seem fit."""
        with open(corpus, 'r') as fp:
            for line in tqdm(fp.readlines()):
                doc_id, document = self.preprocessor.get_doc_id(line)
                tokenized_document = self.preprocessor.tokenizer(document)
                self.indexer.generate_inverted_index(doc_id, tokenized_document)
        self.indexer.sort_terms()
        self.indexer.add_skip_connections()
        self.indexer.calculate_tf_idf()

    def sanity_checker(self, command):
        """ DO NOT MODIFY THIS. THIS IS USED BY THE GRADER. """

        index = self.indexer.get_index()
        kw = random.choice(list(index.keys()))
        return {"index_type": str(type(index)),
                "indexer_type": str(type(self.indexer)),
                "post_mem": str(index[kw]),
                "post_type": str(type(index[kw])),
                "node_mem": str(index[kw].start_node),
                "node_type": str(type(index[kw].start_node)),
                "node_value": str(index[kw].start_node.value),
                "command_result": eval(command) if "." in command else ""}

    def run_queries(self, query_list, random_command):
        """ DO NOT CHANGE THE output_dict definition"""
        output_dict = {'postingsList': {},
                       'postingsListSkip': {},
                       'daatAnd': {},
                       'daatAndSkip': {},
                       'daatAndTfIdf': {},
                       'daatAndSkipTfIdf': {},
                       'sanity': {}#self.sanity_checker(random_command)
                      }

        for query in tqdm(query_list):
            """ Run each query against the index. You should do the following for each query:
                1. Pre-process & tokenize the query.
                2. For each query token, get the postings list & postings list with skip pointers.
                3. Get the DAAT AND query results & number of comparisons with & without skip pointers.
                4. Get the DAAT AND query results & number of comparisons with & without skip pointers, 
                    along with sorting by tf-idf scores."""
            
            input_term_arr = self.preprocessor.tokenizer(query)  # Tokenized query. To be implemented.

            for term in input_term_arr:
                postings,scores = self.indexer.get_index()[term][1].traverse_list()
                skip_postings = self.indexer.get_index()[term][1].traverse_skips()


                """ Implement logic to populate initialize the above variables.
                    The below code formats your result to the required format.
                    To be implemented."""

                output_dict['postingsList'][term] = postings
                output_dict['postingsListSkip'][term] = skip_postings

            and_op_no_skip,and_comparisons_no_skip = self._daat_and(input_term_arr,score=0)
            and_op_skip,and_comparisons_skip = self._daat_and_skip(input_term_arr,score=0) 
            and_op_no_skip_sorted,and_comparisons_no_skip_sorted = self._daat_and(input_term_arr,score=1) 
            and_op_skip_sorted,and_comparisons_skip_sorted = self._daat_and_skip(input_term_arr,score=1) 
            
            """ Implement logic to populate initialize the above variables.
                The below code formats your result to the required format.
                To be implemented."""
            and_op_no_score_no_skip, and_results_cnt_no_skip = self._output_formatter(and_op_no_skip)
            and_op_no_score_skip, and_results_cnt_skip = self._output_formatter(and_op_skip)
            and_op_no_score_no_skip_sorted, and_results_cnt_no_skip_sorted = self._output_formatter(and_op_no_skip_sorted)
            and_op_no_score_skip_sorted, and_results_cnt_skip_sorted = self._output_formatter(and_op_skip_sorted)
            
            output_dict['daatAnd'][query.strip()] = {}
            output_dict['daatAnd'][query.strip()]['results'] = and_op_no_score_no_skip
            output_dict['daatAnd'][query.strip()]['num_docs'] = and_results_cnt_no_skip
            output_dict['daatAnd'][query.strip()]['num_comparisons'] = and_comparisons_no_skip

            output_dict['daatAndSkip'][query.strip()] = {}
            output_dict['daatAndSkip'][query.strip()]['results'] = and_op_no_score_skip
            output_dict['daatAndSkip'][query.strip()]['num_docs'] = and_results_cnt_skip
            output_dict['daatAndSkip'][query.strip()]['num_comparisons'] = and_comparisons_skip

            output_dict['daatAndTfIdf'][query.strip()] = {}
            output_dict['daatAndTfIdf'][query.strip()]['results'] = and_op_no_score_no_skip_sorted
            output_dict['daatAndTfIdf'][query.strip()]['num_docs'] = and_results_cnt_no_skip_sorted
            output_dict['daatAndTfIdf'][query.strip()]['num_comparisons'] = and_comparisons_no_skip_sorted

            output_dict['daatAndSkipTfIdf'][query.strip()] = {}
            output_dict['daatAndSkipTfIdf'][query.strip()]['results'] = and_op_no_score_skip_sorted
            output_dict['daatAndSkipTfIdf'][query.strip()]['num_docs'] = and_results_cnt_skip_sorted
            output_dict['daatAndSkipTfIdf'][query.strip()]['num_comparisons'] = and_comparisons_skip_sorted

        return output_dict

In [54]:
pr = ProjectRunner()
pr

In [55]:
pr.run_indexer("./data/sample_corpus.txt")

100%|█████████████████████████████████████████| 12/12 [00:00<00:00, 5955.00it/s]


In [56]:
pr.indexer.get_index()

OrderedDict([('back', [1, <__main__.LinkedList at 0x7f7db39b88b0>]),
             ('cafe', [1, <__main__.LinkedList at 0x7f7db39c1490>]),
             ('common', [1, <__main__.LinkedList at 0x7f7db39c1f70>]),
             ('even', [1, <__main__.LinkedList at 0x7f7db39c18b0>]),
             ('go', [2, <__main__.LinkedList at 0x7f7db39c12e0>]),
             ('good', [1, <__main__.LinkedList at 0x7f7db39c1d60>]),
             ('health', [1, <__main__.LinkedList at 0x7f7db39c15e0>]),
             ('hello', [5, <__main__.LinkedList at 0x7f7db39ab280>]),
             ('hi', [1, <__main__.LinkedList at 0x7f7d913e0580>]),
             ('jack', [2, <__main__.LinkedList at 0x7f7db39c1670>]),
             ('know', [1, <__main__.LinkedList at 0x7f7db39c1250>]),
             ('let', [1, <__main__.LinkedList at 0x7f7db39c1370>]),
             ('meet', [2, <__main__.LinkedList at 0x7f7db39c1a00>]),
             ('monday', [1, <__main__.LinkedList at 0x7f7db39c1e80>]),
             ('random', [3, <__m

In [57]:
random_command = "random"
query_list = ["hello world",
              "hello swimming", 
              "swimming going", 
              "random swimming"]

In [58]:
pr.run_queries(query_list, random_command)

100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 4938.83it/s]


{'postingsList': {'hello': [1, 2, 5, 9],
  'world': [1, 2],
  'swim': [7, 8, 9],
  'go': [7, 9],
  'random': [10, 11, 12]},
 'postingsListSkip': {'hello': [1, 5],
  'world': [],
  'swim': [7, 9],
  'go': [],
  'random': [10, 12]},
 'daatAnd': {'hello world': {'results': [1, 2],
   'num_docs': 2,
   'num_comparisons': 2},
  'hello swimming': {'results': [9], 'num_docs': 1, 'num_comparisons': 6},
  'swimming going': {'results': [7, 9], 'num_docs': 2, 'num_comparisons': 3},
  'random swimming': {'results': [], 'num_docs': 0, 'num_comparisons': 3}},
 'daatAndSkip': {'hello world': {'results': [1, 2],
   'num_docs': 2,
   'num_comparisons': 2},
  'hello swimming': {'results': [9], 'num_docs': 1, 'num_comparisons': 4},
  'swimming going': {'results': [7, 9], 'num_docs': 2, 'num_comparisons': 3},
  'random swimming': {'results': [], 'num_docs': 0, 'num_comparisons': 2}},
 'daatAndTfIdf': {'hello world': {'results': [1, 2],
   'num_docs': 2,
   'num_comparisons': 2},
  'hello swimming': {'resu